## Dataset

데이터 파일을 불러와서 pandas DataFrame으로 구성하는 방법을 실습해보겠습니다.

우선 학습 데이터를 다운로드하고 압축을 풉니다.

데이터 출처: http://www.cs.cornell.edu/~cristian/Politeness.html

In [0]:
import os

if not os.path.exists("Stanford_politeness_corpus.zip"):
  !wget http://www.cs.cornell.edu/~cristian/Politeness_files/Stanford_politeness_corpus.zip

if not os.path.exists("Stanford_politeness_corpus/wikipedia.annotated.csv"):
  !unzip Stanford_politeness_corpus.zip

다운로드된 데이터셋은 CSV 파일 형태로 되어 있습니다.

파일이 어떻게 구성되어 있는지 살펴볼까요?

In [0]:
!head Stanford_politeness_corpus/wikipedia.annotated.csv

한 줄이 하나의 데이터를 나타내고, 각 데이터 값들은 쉼표로 구분되어 있는 것을 볼 수 있습니다.

또한, 첫 줄은 각 데이터 값들이 어떤 항목을 나타내는지 알려주는 헤더입니다.

이런 형식의 데이터 파일은 pandas의 read_csv 함수를 이용하여 쉽게 불러올 수 있습니다.

참고: [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [0]:
import urllib
import pandas as pd

data = pd.read_csv("Stanford_politeness_corpus/wikipedia.annotated.csv")
pd.set_option('display.max_columns', None)

print(data.head())

이번엔 다른 형태의 데이터를 다운로드 받아보겠습니다. [GLUE dataset](https://gluebenchmark.com) 중 하나인 Stanford Sentiment Treebank 입니다.

In [0]:
import os
import urllib

download_url = "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8"

if not os.path.exists("sst.zip"):
  urllib.request.urlretrieve(download_url, "sst.zip")

if not os.path.exists("SST-2/train.tsv"):
  !unzip -o sst.zip
  
!head SST-2/train.tsv

이번에도 한 줄이 하나의 데이터를 나타냅니다. 

하지만 이번에는 데이터 항목들 간의 구분자가 쉼표가 아닌 탭입니다.

pandas.read_csv 함수는 sep 파라미터를 이용하여 구분자를 지정해줄 수 있습니다. 앞선 사용 예에서는 sep이 지정되지 않았는데, 이 때는 쉼표가 기본값으로 사용됩니다.

다시 한 번 pandas.read_csv 함수를 이용하여 이 데이터셋을 불러와보겠습니다.

In [0]:
data = pd.read_csv("SST-2/train.tsv", sep='\t')
pd.set_option('display.max_columns', None)

print(data.head())

파일 파싱 에러(ParseError) 등으로 인해 read_csv 함수를 사용할 수 없는 경우도 있습니다. 

그런 경우는 우선 python을 이용하여 데이터를 배열 형태로 불러오고, 이로부터 pandas DataFrame을 생성할 수 있습니다.

Microsoft Research Paraphrase Corpus(MRPC)를 이용하여 이러한 경우를 실습해보겠습니다.

MRPC는 저작권 문제로 인해 마이크로소프트 홈페이지에서 다운로드를 받아야 합니다. 다음 링크에서 데이터를 다운로드 받아주세요.

https://www.microsoft.com/en-us/download/details.aspx?id=52398

다운로드 완료 후에는 좌측의 파일 업로드 기능을 이용하여 해당 파일을 업로드해주시기 바랍니다.

해당 파일은 msi 형태로, 별도의 프로그램을 통해 압축을 풀고 스크립트를 이용해 전처리를 해야 합니다.

In [0]:
import os

if not os.path.exists("MRPC/msr_paraphrase_train.txt"):
  !apt install cabextract

  !cabextract MSRParaphraseCorpus.msi -d MRPC
  !cat MRPC/_2DEC3DBE877E4DB192D17C0256E90F1D | tr -d $'\r' > MRPC/msr_paraphrase_train.txt
  !cat MRPC/_D7B391F9EAFF4B1B8BCE8F21B20B1B61 | tr -d $'\r' > MRPC/msr_paraphrase_test.txt
  !rm MRPC/_*
  
!head MRPC/msr_paraphrase_train.txt

다음으로 python으로 파일을 읽고 각 줄을 탭을 기준으로 분리하는 과정을 통해 DataFrame을 생성해보겠습니다.

이 경우에는 columns 파라미터를 통해 헤더를 직접 지정해줘야 합니다.

참고: [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)

In [0]:
with open("MRPC/msr_paraphrase_train.txt", 'r') as fh:
  raw_data = [line.strip().split('\t') for line in fh]
  
data = pd.DataFrame(raw_data[1:], columns=raw_data[0])
  
print(data.head())

## BERT를 이용한 Text Classification

BERT를 이용한 text classification 모델을 만들고 학습시켜보겠습니다.

우선 필요한 패키지를 설치해야 합니다. bert-tensorflow 패키지는 구글에서 공개한 BERT의 코드로부터 BERT 사용에 필요한 함수들을 활용할 수 있게 해줍니다.

참고: [BERT github](https://github.com/google-research/bert)

In [0]:
! pip install bert-tensorflow

In [0]:
%tensorflow_version 1.x
import math
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import bert
import os
from bert import run_classifier
from bert import optimization
from bert import tokenization

tf.logging.set_verbosity(tf.logging.ERROR)


def create_tokenizer_from_hub_module(bert_model_hub):
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(bert_model_hub)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                  tokenization_info["do_lower_case"]])

        print("Using BERT from %s" %bert_model_hub)
        print("with vocab size=%d and do_lower_case=%s." %(len(vocab_file), str(do_lower_case)))

    return bert.tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)


def make_features(dataset, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN):
    input_example = dataset.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                             text_a=x[DATA_COLUMN],
                                                                             text_b=None,
                                                                             label=x[LABEL_COLUMN]), axis=1)
    features = bert.run_classifier.convert_examples_to_features(input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
    return features


def create_model(bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
    """Creates a classification model."""

    bert_module = hub.Module(
        bert_model_hub,
        trainable=True)
    bert_inputs = dict(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids)
    bert_outputs = bert_module(
        inputs=bert_inputs,
        signature="tokens",
        as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    with tf.variable_scope("output_layer"):
        layer_out = tf.layers.dense(
            inputs=output_layer,
            units=num_labels,
            use_bias=False,
            kernel_initializer=tf.initializers.variance_scaling()
        )
        predicted_labels = tf.squeeze(tf.argmax(layer_out, axis=-1, output_type=tf.int32))

        if is_predicting:
            return predicted_labels, layer_out
        else:
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=labels,
                logits=layer_out
            )
            loss = tf.reduce_mean(loss)

            return loss, predicted_labels, layer_out


# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(bert_model_hub, num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(
                bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(
                loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics.
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(
                    label_ids,
                    predicted_labels)
                auc = tf.metrics.auc(
                    label_ids,
                    predicted_labels)
                recall = tf.metrics.recall(
                    label_ids,
                    predicted_labels)
                precision = tf.metrics.precision(
                    label_ids,
                    predicted_labels)
                true_pos = tf.metrics.true_positives(
                    label_ids,
                    predicted_labels)
                true_neg = tf.metrics.true_negatives(
                    label_ids,
                    predicted_labels)
                false_pos = tf.metrics.false_positives(
                    label_ids,
                    predicted_labels)
                false_neg = tf.metrics.false_negatives(
                    label_ids,
                    predicted_labels)
                return {
                    "eval_accuracy": accuracy,
                    "f1_score": f1_score,
                    "auc": auc,
                    "precision": precision,
                    "recall": recall,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,
                                                  loss=loss,
                                                  train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode,
                                                  loss=loss,
                                                  eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
                bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {
                'probabilities': log_probs,
                'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn


def estimator_builder(bert_model_hub, OUTPUT_DIR, SAVE_SUMMARY_STEPS, SAVE_CHECKPOINTS_STEPS, label_list, LEARNING_RATE,
                      num_train_steps, num_warmup_steps, BATCH_SIZE):
    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    model_fn = model_fn_builder(
        bert_model_hub=bert_model_hub,
        num_labels=len(label_list),
        learning_rate=LEARNING_RATE,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        config=run_config,
        params={"batch_size": BATCH_SIZE})
    return estimator, model_fn, run_config


def run_on_dfs(train, test, data_column, label_column,
               max_seq_length=128,
               batch_size=32,
               learning_rate=2e-5,
               num_train_epochs=3,
               warmup_proportion=0.1,
               save_summary_steps=100,
               save_checkpoint_steps=10000,
               bert_model_hub="https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1",
               output_dir="output"):
    label_list = train[label_column].unique().tolist()

    tokenizer = create_tokenizer_from_hub_module(bert_model_hub)

    train_features = make_features(train, label_list, max_seq_length, tokenizer, data_column, label_column)
    test_features = make_features(test, label_list, max_seq_length, tokenizer, data_column, label_column)

    steps_per_epoch = math.ceil(len(train_features) / batch_size)

    num_train_steps = int(len(train_features) / batch_size * num_train_epochs)
    num_warmup_steps = int(num_train_steps * warmup_proportion)

    estimator, model_fn, run_config = estimator_builder(
        bert_model_hub,
        output_dir,
        save_summary_steps,
        save_checkpoint_steps,
        label_list,
        learning_rate,
        num_train_steps,
        num_warmup_steps,
        batch_size)

    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=max_seq_length,
        is_training=True,
        drop_remainder=False)

    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=max_seq_length,
        is_training=False,
        drop_remainder=False)

    results = []
    for epoch in range(num_train_epochs):
        estimator.train(input_fn=train_input_fn, steps=steps_per_epoch)

        print("End of epoch %d." %(epoch + 1))

        result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)
        print(result_dict)
        results.append(result_dict)

    return results, estimator


def pretty_print(result):
    df = pd.DataFrame([result]).T
    df.columns = ["values"]
    return df

In [0]:
def load_data(data_file):
    data = pd.read_csv(data_file)

    # Only use the top quartile as polite, and bottom quartile as impolite. Discard the rest.
    quantiles = data["Normalized Score"].quantile([0.25, 0.5, 0.75])
    # print(quantiles)

    for i in range(len(data)):
        score = data.loc[i, "Normalized Score"]
        if score <= quantiles[0.25]:
            # Bottom quartile (impolite).
            data.loc[i, "Normalized Score"] = 0
        elif score >= quantiles[0.75]:
            # Top quartile (polite).
            data.loc[i, "Normalized Score"] = 1
        else:
            # Neutral.
            data.loc[i, "Normalized Score"] = 2

    data["Normalized Score"] = data["Normalized Score"].astype(int)

    # Discard neutral examples.
    data = data[data["Normalized Score"] < 2]
    
    data.sample(frac=1).reset_index(drop=True)
    n_test = len(data) // 10
    test_data = data[:n_test]
    train_data = data[n_test:]
    
    print("Data loaded successfully. Train=%d, test=%d, total=%d." % (len(train_data), len(test_data), len(train_data) + len(test_data)))
    print("Some train samples:")
    print(train_data.head())
    print("Some test samples:")
    print(test_data.head())

    return train_data, test_data

In [0]:
if not os.path.exists("Stanford_politeness_corpus.zip"):
  !wget http://www.cs.cornell.edu/~cristian/Politeness_files/Stanford_politeness_corpus.zip

if not os.path.exists("Stanford_politeness_corpus/wikipedia.annotated.csv"):
  !unzip Stanford_politeness_corpus.zip

train_data, test_data = load_data("Stanford_politeness_corpus/wikipedia.annotated.csv")

params = {
    "data_column": "Request",
    "label_column": "Normalized Score",
    "batch_size": 16,
    "num_train_epochs": 3,
    "bert_model_hub": "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"
}

tf.logging.set_verbosity(tf.logging.INFO)
result, estimator = run_on_dfs(train_data, test_data, **params)
print(result)

## Tensorboard를 이용한 학습 결과 시각화

[Tensorboard](https://www.tensorflow.org/tensorboard)는 TensorFlow와 함께 제공되는 시각화 도구입니다.
단순히 학습 중 loss 함수의 값, 성능 뿐만이 아니라 사진, 글, 임베딩 등 다양한 정보들을 기록하고, 확인할 수 있습니다.

앞서 실습한 코드에서는 이미 Tensorboard를 위한 데이터를 기록하도록 되어 있습니다. 좌측 설정에서 파일들을 보시면 "output"이라는 폴더가 생겼고, 그 아래 "events.out.tfevents"로 시작하는 파일이 생성된 것을 보실 수 있습니다. 이 파일에 Tensorboard를 위한 정보가 담겨있습니다.

Colab에서 Tensorboard를 확인하려면 아래 코드를 실행하면 됩니다.


In [0]:
%load_ext tensorboard
%tensorboard --logdir output